In [1]:
import numpy as np  
import numpy.polynomial.legendre as npl
import sympy as sp
from scipy.special import legendre

In [2]:
def Transformation(nodes, weight,a,b):
    nodes=(b-a)/2*nodes+(a+b)/2
    weight=(b-a)/2 *weight
    return (nodes,weight)

In [3]:
def chop(lista):
    for i in range(len(lista)):
        if(abs(lista[i])<(10**(-10))):
            lista[i]=0
    return lista

In [3]:
def Derivate(f,nodes):
    d=np.zeros(len(nodes))
    for i in range(0,len(nodes)):
        if i==0:
            d[i]=(f(nodes[i+1])-f(nodes[i]))/(nodes[i+1]-nodes[i])
        elif i==(len(nodes)-1):
            d[i]=(f(nodes[i])-f(nodes[i-1]))/(nodes[i]-nodes[i-1])
        else:
            d[i]=(f(nodes[i+1])-f(nodes[i-1]))/(nodes[i+1]-nodes[i-1])
    return d

Probando que funciona la Derivada por la definición de $\frac{f(x+h)-f(x)}{h}$

In [4]:
f= lambda x: x**2
nodes=[0,0.1,0.2,0.25,0.3]

Derivate(f,nodes)

array([0.1 , 0.2 , 0.35, 0.5 , 0.55])

In [17]:
N=4
x = sp.symbols('x')
Ln = lambda x:chop(legendre(4))(x)
Lnp= lambda x:chop(legendre(4)).deriv()(x)
xvals = np.linspace(-1,1,N+1)

phi0 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[[0]])*(Ln(xvals)[[0]]))))
difphi0 = lambda x: sp.simplify(sp.diff(phi0(x)))
difphi0 = lambda x: 3.5*x**3-2.625*x**2-0.75*x+0.375

phi1 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[[1]])*(Ln(xvals)[[1]]))))
difphi1 = lambda x: sp.simplify(sp.diff(phi1(x)))
difphi1 = lambda x: (-3.5*x**5-2.1875*x**4+2.5*x**3+1.875*x**2-0.1875)/(0.2890625*x**2+0.2890625*x+0.072265625)

phi2 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[[2]])*(Ln(xvals)[[2]]))))
difphi2 = lambda x: sp.simplify(sp.diff(phi2(x)))
difphi2 = lambda x: x*(9.33333333333333*x**2-6.66666666666667)

phi3 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[[3]])*(Ln(xvals)[[3]]))))
difphi3 = lambda x: sp.simplify(sp.diff(phi3(x)))
difphi3 = lambda x: (-3.5*x**5+2.1875*x**4+2.5*x**3-1.875*x**2+0.1875)/(0.2890625*x**2-0.2890625*x+0.072265625)

phi4 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[[4]])*(Ln(xvals)[[4]]))))
difphi4 = lambda x: sp.simplify(sp.diff(phi4(x)))
difphi4 = lambda x: 3.5*x**3+2.625*x**2-0.75*x-0.375

## Hay problemas con los infinitos

In [22]:
print(difphi3(xvals),difphi4(xvals))

[ 2.30630631 -1.2027027   2.59459459         inf -6.91891892] [-0.5      0.21875 -0.375    0.34375  5.     ]


C:\Users\isabe\Desktop\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in true_divide


In [20]:
difphi4(xvals)*difphi3(xvals)

C:\Users\isabe\Desktop\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in true_divide


array([ -1.15315315,  -0.26309122,  -0.97297297,          inf,
       -34.59459459])

In [24]:
class Cuadrature:
    def __init__(self,nodes=None,weight=None):
        if(list(nodes)!=None):
                self.nodes=nodes
        if(list(weight)!=None):
                self.weight=weight
        
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def getNumPoints(self):
        return len(self.nodes)
    
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")

Pasar todos los self.nodes y self.weight arriba poniendo num_points.

Intentar integrar bien las funciones o su funcion derivada.

In [25]:
class Legendre(Cuadrature):
    def __init__(self,num_points,a,b):
        Cuadrature.__init__(self,nodes=npl.leggauss(num_points)[0], weight=npl.leggauss(num_points)[1]) 
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [26]:
class Trapecio_Compuesto(Cuadrature): #están considerados los extremos(-1,1)
    def __init__(self,num_points,a,b):
        Cuadrature.__init__(self, nodes=np.array([*np.arange(-1,1,float(2/num_points)).tolist(),1]), weight=np.array([2/(2*num_points),*[2/num_points]*(num_points-1),2/(2*num_points)]))
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [27]:
class Lobato(Cuadrature):
    def __init__(self, num_points,a,b):
        aux=[-1,*sorted(list(legendre(num_points).deriv().r)),1]
        Cuadrature.__init__(self, nodes=np.array(aux), weight=np.array(2/(num_points*(num_points+1)*(legendre(num_points)(aux))**2)))
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [29]:
print(Legendre(3,-1,1))

Nodos: [-0.77459667  0.          0.77459667] 
Pesos: [0.55555556 0.88888889 0.55555556] 


In [30]:
print(Legendre(3,0,3))

Nodos: [0.338105 1.5      2.661895] 
Pesos: [0.83333333 1.33333333 0.83333333] 


In [31]:
print(Trapecio_Compuesto(5,0,1))

Nodos: [0.  0.2 0.4 0.6 0.8 1. ] 
Pesos: [0.1 0.2 0.2 0.2 0.2 0.1] 


In [32]:
print(Lobato(3,0,5))

Nodos: [0.         1.38196601 3.61803399 5.        ] 
Pesos: [0.41666667 2.08333333 2.08333333 0.41666667] 


Veamos que integra correctamente:

In [42]:
f= lambda x: (x**2+1)*(2*x+3)

a=2
b=3

Legendre(10,a,b).Integrate(f)

59.49999999999999

In [43]:
Lobato(3,a,b).Integrate(f)

59.50000000000001

In [46]:
xvals3 = np.linspace(a,b,4)

In [52]:
Legendre(10,a,b).Integrate_List(f(np.linspace(a,b,10)))

59.162198819932584

In [56]:
Lobato(10,a,b).Integrate_List(f(np.linspace(a,b,11)))

58.93094961484155

Para cada l = 1...m siendo m intervalos (m=5):

Vamos a construir la matriz local:

Para cada i=0..p (p=4)
    
  Para cada j=0...p
    
   ($A_l)_{ij}=\sum_{q=1}^{Nq} w_q J_{x_l}(x_q) \varphi_i'(x_q)\varphi_j'(x_q)$
   (Nq=8)

In [ ]:
m=5
intervals=np.linspace(-1,1,m+1)

In [77]:
def Matriz_Local(m,p,Nq,num_interval):
    
    intervals=np.linspace(-1,1,m+1)
    A=np.zeros((p+1, p+1))
    a=intervals[num_interval-1]
    b=intervals[num_interval]
    xvals2=np.linspace(a,b,Nq)
    
    A[0][0]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi0(xvals2))
    A[0][1]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi1(xvals2))
    A[0][2]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi2(xvals2))
    A[0][3]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi3(xvals2))
    A[0][4]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi4(xvals2))

    A[1][0]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi0(xvals2))
    A[1][1]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi1(xvals2))
    A[1][2]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi2(xvals2))
    A[1][3]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi3(xvals2))
    A[1][4]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi4(xvals2))

    A[2][0]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi0(xvals2))
    A[2][1]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi1(xvals2))
    A[2][2]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi2(xvals2))
    A[2][3]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi3(xvals2))
    A[2][4]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi4(xvals2))

    A[3][0]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi0(xvals2))
    A[3][1]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi1(xvals2))
    A[3][2]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi2(xvals2))
    A[3][3]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi3(xvals2))
    A[3][4]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi4(xvals2))

    A[4][0]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi0(xvals2))
    A[4][1]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi1(xvals2))
    A[4][2]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi2(xvals2))
    A[4][3]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi3(xvals2))
    A[4][4]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi4(xvals2))
    
    return A

In [84]:
A1=Matriz_Local(7,4,3,1) #m=7 intervalos dentro del (-1,1)
A2=Matriz_Local(7,4,3,2) #las phi siguen siendo del 0 al 4 (5) grados de libertad
A3=Matriz_Local(7,4,3,3) #Consideramos 3 nodos de cuadratura
A4=Matriz_Local(7,4,3,4)
A5=Matriz_Local(7,4,3,5)
A6=Matriz_Local(7,4,3,6)
A7=Matriz_Local(7,4,3,7)

In [85]:
A1

array([[ 3.17674803, -2.15023291,  0.6213813 , -0.50130904,  0.10537152],
       [-2.15023291,  3.68491396, -1.22016217,  1.01330519, -0.21568762],
       [ 0.6213813 , -1.22016217,  0.43184828, -0.36487401,  0.07827426],
       [-0.50130904,  1.01330519, -0.36487401,  0.30961424, -0.06654742],
       [ 0.10537152, -0.21568762,  0.07827426, -0.06654742,  0.01431571]])

In [76]:
p=4
A1 = np.zeros((p+1, p+1))

Nq=8
a=intervals[0]
b=intervals[1]
xvals2=np.linspace(a,b,8)
A1[0][0]=Lobato(7,a,b).Integrate_List(difphi0(xvals2)*difphi0(xvals2))
A1[0][1]=Lobato(7,a,b).Integrate_List(difphi0(xvals2)*difphi1(xvals2))
A1[0][2]=Lobato(7,a,b).Integrate_List(difphi0(xvals2)*difphi2(xvals2))
A1[0][3]=Lobato(7,a,b).Integrate_List(difphi0(xvals2)*difphi3(xvals2))
A1[0][4]=Lobato(7,a,b).Integrate_List(difphi0(xvals2)*difphi4(xvals2))

A1[1][0]=Lobato(7,a,b).Integrate_List(difphi1(xvals2)*difphi0(xvals2))
A1[1][1]=Lobato(7,a,b).Integrate_List(difphi1(xvals2)*difphi1(xvals2))
A1[1][2]=Lobato(7,a,b).Integrate_List(difphi1(xvals2)*difphi2(xvals2))
A1[1][3]=Lobato(7,a,b).Integrate_List(difphi1(xvals2)*difphi3(xvals2))
A1[1][4]=Lobato(7,a,b).Integrate_List(difphi1(xvals2)*difphi4(xvals2))

A1[2][0]=Lobato(7,a,b).Integrate_List(difphi2(xvals2)*difphi0(xvals2))
A1[2][1]=Lobato(7,a,b).Integrate_List(difphi2(xvals2)*difphi1(xvals2))
A1[2][2]=Lobato(7,a,b).Integrate_List(difphi2(xvals2)*difphi2(xvals2))
A1[2][3]=Lobato(7,a,b).Integrate_List(difphi2(xvals2)*difphi3(xvals2))
A1[2][4]=Lobato(7,a,b).Integrate_List(difphi2(xvals2)*difphi4(xvals2))

A1[3][0]=Lobato(7,a,b).Integrate_List(difphi3(xvals2)*difphi0(xvals2))
A1[3][1]=Lobato(7,a,b).Integrate_List(difphi3(xvals2)*difphi1(xvals2))
A1[3][2]=Lobato(7,a,b).Integrate_List(difphi3(xvals2)*difphi2(xvals2))
A1[3][3]=Lobato(7,a,b).Integrate_List(difphi3(xvals2)*difphi3(xvals2))
A1[3][4]=Lobato(7,a,b).Integrate_List(difphi3(xvals2)*difphi4(xvals2))

A1[4][0]=Lobato(7,a,b).Integrate_List(difphi4(xvals2)*difphi0(xvals2))
A1[4][1]=Lobato(7,a,b).Integrate_List(difphi4(xvals2)*difphi1(xvals2))
A1[4][2]=Lobato(7,a,b).Integrate_List(difphi4(xvals2)*difphi2(xvals2))
A1[4][3]=Lobato(7,a,b).Integrate_List(difphi4(xvals2)*difphi3(xvals2))
A1[4][4]=Lobato(7,a,b).Integrate_List(difphi4(xvals2)*difphi4(xvals2))

A1

array([[ 3.05003153e+00, -2.69366332e-01,  3.42020675e-02,
        -1.19744778e-02,  9.45263963e-04],
       [-2.69366332e-01,  1.04939086e+01, -2.06459147e+00,
         1.58248311e+00, -3.25235027e-01],
       [ 3.42020675e-02, -2.06459147e+00,  5.18781471e-01,
        -4.11821518e-01,  8.58805375e-02],
       [-1.19744778e-02,  1.58248311e+00, -4.11821518e-01,
         3.28833285e-01, -6.87467329e-02],
       [ 9.45263963e-04, -3.25235027e-01,  8.58805375e-02,
        -6.87467329e-02,  1.43879501e-02]])

# Pasos que he seguido a la hora de construir las clases

## Legendre:
Los nodos y pesos de Legendre se obtienen directamente con la función

In [17]:
npl.leggauss(3)

(array([-0.77459667,  0.        ,  0.77459667]),
 array([0.55555556, 0.88888889, 0.55555556]))

## Lobato:
En la fórmula de Gauss_Lobato los nodos interiores son los ceros de la derivada del polinomio de Legendre $L_N'(x_i)=0$ y los pesos los he sacado de la fórmula del libro de Quarteroni.

In [41]:
n=2 #son n+1 puntos en total del x_0 al x_n

interior=legendre(n).deriv().r
nodos=[-1,*sorted(list(interior)),1]

pesos=2/(n*(n+1)*(legendre(n)(nodos))**2)

print(nodos,pesos)

[-1, 0.0, 1] [0.33333333 1.33333333 0.33333333]


In [43]:
legendre(3)(nodos)

array([-1.,  0.,  1.])

In [22]:
legendre(n)(nodos)

array([ 1. , -0.5,  1. ])

## Trapecios Compuestos:
En la fórmula de los trapecios he considerado el intervalo $(-1,1)$ en $n$ intervalos, de modo que tengo $n+1$ puntos, el incremento y por tanto h es, $h=\frac{b-a}n$ y los pesos en los extremos son $\frac h 2$ y en los nodos interiores $h$.

In [19]:
a,b=-1,1
n=6
[*np.arange(-1,1,float((b-a)/n)).tolist(),1] #nodos en Trapecios

[-1.0,
 -0.6666666666666667,
 -0.3333333333333335,
 -2.220446049250313e-16,
 0.33333333333333304,
 0.6666666666666663,
 1]

In [20]:
n=6 #x_0,...x_6 7 puntos
[2/(2*n),*[2/n]*(n-1),2/(2*n)] #pesos en Trapecios

[0.16666666666666666,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.16666666666666666]

## Con Dani y Rafa Lunes 14 Marzo

In [21]:
class Cuadrature:
    def __init__(self,nodes=None,weight=None,num_points=None):
        if (nodes!=None and weight!=None):
            self.nodes = nodes
            self.weight = weight
            print(self.nodes,self.weight)
        elif(num_points!=None):
            self.nodes, self.weight= npl.leggauss(num_points)
            print(self.nodes,self.weight)
        else:
            raise Exception("No has metido correctamente los parámetros en la clase")
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")
        

In [22]:
Cuadrature(4)

Exception: No has metido correctamente los parámetros en la clase

In [25]:
cuad_gauss=Cuadrature(num_points=6)

[-0.93246951 -0.66120939 -0.23861919  0.23861919  0.66120939  0.93246951] [0.17132449 0.36076157 0.46791393 0.46791393 0.36076157 0.17132449]


In [26]:
f= lambda x: x**10
cuad_gauss.Integrate(f)

0.18181818181818124

In [27]:
lista=[2,4,5,0]
cuad_gauss.Integrate_List(lista)

Exception: No has metido correctamente la lista